In [ ]:
Loads the .vcf files (Snakemake output) and generates a annotated rds dataset

Requires:
- pipeline_data/vcf_wgs_130721/*.vcf
- datasets/contig_mapping_240621.csv
- datasets/gff3_from_genbank_220621/*.gff3"

In [11]:
setwd("../../")

In [2]:
install.packages("vcfR")

also installing the dependencies ‘permute’, ‘cluster’, ‘ape’, ‘memuse’, ‘pinfsc50’, ‘vegan’


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [4]:
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("rtracklayer")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.r-project.org


Bioconductor version 3.14 (BiocManager 1.30.16), R 4.1.3 (2022-03-10)

Installing package(s) 'BiocVersion', 'rtracklayer'

also installing the dependencies ‘formatR’, ‘lambda.r’, ‘futile.options’, ‘matrixStats’, ‘futile.logger’, ‘snow’, ‘MatrixGenerics’, ‘Biobase’, ‘DelayedArray’, ‘GenomeInfoDbData’, ‘BiocParallel’, ‘Rhtslib’, ‘SummarizedExperiment’, ‘rjson’, ‘GenomicRanges’, ‘XML’, ‘BiocGenerics’, ‘S4Vectors’, ‘IRanges’, ‘XVector’, ‘GenomeInfoDb’, ‘Biostrings’, ‘zlibbioc’, ‘Rsamtools’, ‘GenomicAlignments’, ‘BiocIO’, ‘restfulr’


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Old packages: 'blob', 'gert', 'later', 'parallelly', 'randomForest', 'RSQLite'



In [12]:
#source("utils/helpers.R")
library(rtracklayer)
library(data.table)
library(tidyr)
library(vcfR)

In [19]:
library(OligoMMR2)

Parameters

In [13]:
qual_threshold <- 0 # removes SNPs that have a value below qual_threshold in the QUAL field in the .vcf file
vcf_files <- Sys.glob("pipeline_data/vcf/vcf_wgs_130721/*.vcf") # Path to vcf files
gff_files <- Sys.glob("datasets/gff3_from_genbank_220621/*.gff3") # Path to annotaiton of reference genomes
contig_mapping_path <- "datasets/contig_mapping_240621.csv" # Path to mapping file

Imports a vcf file and returns the annotated data frame
vcf_files Path to vcf file
ontig_mapping Table with mapping of contig to genome ID
gff_df Dataframe containing the genome annotation e.g. produced by Prokka
Returns A dataframe containing annotation fields

In [14]:
vcfToDataframe <- function(vcf_files, contig_mapping = contig_mapping, gff_df = gff_df) {
  res <- list()
  for (vcf.file in vcf_files) {
    message(vcf.file)
    vcf.content <- vcfR::read.vcfR(vcf.file, verbose = FALSE)
    vcf.fix <- as.data.frame(vcf.content@fix) # contains chr, position and substitution informations
    vcf.info <- vcfR::INFO2df(vcf.content) # get INFO field, contains DP, AF informations
    if(nrow(vcf.fix) > 0) { # there are variants
      dat <-  as.data.frame(cbind(vcf.fix[,c(1, 2, 4, 5, 6)], vcf.info[,c(1, 2)]))
      dat$majorAF <- sapply(dat$AF, minorAfToMajorAf)
      dat$genome <- contig_mapping[match(dat$CHROM, contig_mapping$contig),]$genome
      dat$genome_hr <- translateGenomeIdToFullName(tolower(dat$genome))
      dat$mouse.id <-  substr(tools::file_path_sans_ext(basename(vcf.file)), 1, 4)
      dat$mouse.group <- translateMouseIdToTreatmentGroup(dat$mouse.id)
      dat$day <- as.integer(substr(basename(vcf.file), 6, 7))
      dat$phase <- binDaysByPhase(as.numeric(as.matrix(dat$day)))
      dat$phase_num <- binDaysByPhaseGroup(dat$day)
      dat$dp <- as.numeric(as.matrix(vcf.info$DP))
      # annotate overlap with a ORF
      dt.gff <- data.table(start = gff_df$start, end = gff_df$end,
                           chr = as.character(as.matrix(gff_df$chr)), feature = gff_df$product)
      colnames(dat)[1:2] <- c("chr", "start")
      dat$start <- as.integer(as.matrix(dat$start))
      dat$chr <- as.character(as.matrix(dat$chr))
      dat$end <- dat$start
      dat2 <- as.data.table(dat)
      setkey(dt.gff, chr, start, end)
      annotated <- foverlaps(dat2, dt.gff, type = "within", mult = "first")
      res[[tools::file_path_sans_ext(basename(vcf.file))]] <- annotated
    } else{
      message(paste("Skipping", file))
    }
  }
  df <- as.data.frame(do.call(rbind, res)) # merge list to df
  return(df)
}

In [15]:
gff_df <- NULL
for (gff.file in gff_files) {

  gff <- rtracklayer::readGFF(gff.file)
  relevant <- data.frame(start = gff$start, end = gff$end, type = as.character(as.matrix(gff$type)), gene = as.character(as.matrix(gff$gene)), product = as.character(as.matrix(gff$product)), chr = as.character(as.matrix(gff$seqid)))
  relevant$genome <-  substr(basename(gff.file), 1, nchar(basename(gff.file)) - 5)
  gff_df <- rbind(gff_df, relevant)
  gff_df <- gff_df[which( gff_df$type== "CDS"),]
}

dim(gff_df)
head(gff_df)

[1] 41101     7

,start,end,type,gene,product,chr,genome
,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
3,403,675,CDS,NA,type II toxin-antitoxin system YafQ family toxin,Acutalibacter_muris_KB18,Acutalibacter_muris
5,668,910,CDS,NA,hypothetical protein,Acutalibacter_muris_KB18,Acutalibacter_muris
7,1284,2432,CDS,NA,site-specific integrase,Acutalibacter_muris_KB18,Acutalibacter_muris
9,2562,3911,CDS,NA,hypothetical protein,Acutalibacter_muris_KB18,Acutalibacter_muris
11,3914,4321,CDS,NA,hypothetical protein,Acutalibacter_muris_KB18,Acutalibacter_muris
13,4644,5006,CDS,NA,helix-turn-helix domain-containing protein,Acutalibacter_muris_KB18,Acutalibacter_muris


In [17]:
contig_mapping <- read.csv2(contig_mapping_path, sep = ";",
                            header = T, stringsAsFactors = F)
head(contig_mapping)

,contig,genome
,<chr>,<chr>
1,Akkermansia_muciniphila_YL44,YL44
2,Acutalibacter_muris_KB18,KB18
3,Bifidobacterium_animalis_YL2,YL2
4,Bacteroides_caecimuris_I48,I48
5,Blautia_coccoides_YL58,YL58
6,Clostridium_innocuum_I46,I46


In [21]:
minorAfToMajorAf <- function(x) if (x > 0) max(1 - x, x) else NA

In [22]:
vcf_samples <- vcfToDataframe(vcf_files, contig_mapping, gff_df = gff_df)
vcf_samples$feature <- as.character(as.matrix(vcf_samples$feature))
vcf_samples[which(is.na(vcf_samples$feature)),]$feature <- "outside ORFs"
vcf_samples$start <- NULL
vcf_samples$end <- NULL
vcf_samples$i.end <- NULL
colnames(vcf_samples)[3] <- "POS"
vcf_samples$ref_size <- nchar(as.character(as.matrix(vcf_samples$REF)))
vcf_samples$alt_size <- nchar(as.character(as.matrix(vcf_samples$ALT)))
vcf_samples$alteration <- paste(as.character(vcf_samples$REF), "->",as.character(vcf_samples$ALT))
vcf_samples$alteration_type <- "SNP"
vcf_samples$QUAL <- as.integer(as.matrix(vcf_samples$QUAL))
vcf_samples <- vcf_samples[which(vcf_samples$QUAL >= qual_threshold),]
vcf_samples$snp_id <- paste0(vcf_samples$POS, "-", vcf_samples$REF, "-", vcf_samples$ALT)
head(vcf_samples)

pipeline_data/vcf/vcf_wgs_130721/1681d00_S56.vcf

Loading required package: dplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:GenomicRanges’:

    intersect, setdiff, union


The following object is masked from ‘package:GenomeInfoDb’:

    intersect


The following objects are masked from ‘package:IRanges’:

    collapse, desc, intersect, setdiff, slice, union


The following objects are masked from ‘package:S4Vectors’:

    first, intersect, rename, setdiff, setequal, union


The following objects are masked from ‘package:BiocGenerics’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


pipeline_data/vcf/vcf_wgs_130721/1681d14_S57.vcf

pipeline_data/vcf/vcf_wgs_130721/1681d30_S58.vcf

pipeline_data/vcf/vcf

,chr,feature,POS,REF,ALT,QUAL,DP,AF,majorAF,genome,⋯,mouse.group,day,phase,phase_num,dp,ref_size,alt_size,alteration,alteration_type,snp_id
,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<chr>,⋯,<chr>,<int>,<chr>,<dbl>,<dbl>,<int>,<int>,<chr>,<chr>,<chr>
1,Akkermansia_muciniphila_YL44,autotransporter-associated beta strand repeat-containing protein,239850,C,T,1402,129,0.379845,0.620155,YL44,⋯,Control,0,pre-treatment,1,129,1,1,C -> T,SNP,239850-C-T
2,Akkermansia_muciniphila_YL44,autotransporter-associated beta strand repeat-containing protein,240383,A,G,1086,116,0.344828,0.655172,YL44,⋯,Control,0,pre-treatment,1,116,1,1,A -> G,SNP,240383-A-G
3,Akkermansia_muciniphila_YL44,autotransporter-associated beta strand repeat-containing protein,241793,A,G,116,129,0.046512,0.953488,YL44,⋯,Control,0,pre-treatment,1,129,1,1,A -> G,SNP,241793-A-G
4,Akkermansia_muciniphila_YL44,PEP-CTERM sorting domain-containing protein,296139,C,T,1067,160,0.250000,0.750000,YL44,⋯,Control,0,pre-treatment,1,160,1,1,C -> T,SNP,296139-C-T
5,Akkermansia_muciniphila_YL44,protein kinase,355082,A,G,1281,130,0.353846,0.646154,YL44,⋯,Control,0,pre-treatment,1,130,1,1,A -> G,SNP,355082-A-G
6,Akkermansia_muciniphila_YL44,protein kinase,355082,A,T,950,130,0.276923,0.723077,YL44,⋯,Control,0,pre-treatment,1,130,1,1,A -> T,SNP,355082-A-T


In [25]:
saveRDS(vcf_samples, file = "R/datasets/omm_wgs_rerun.rds")